In [1]:
import pandas as pd
import numpy as np 



In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
data = pd.read_csv("C:/Users/dhruv/Documents/DS-class/classification/soh/Youtube05-Shakira.csv")

In [4]:
data.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z13lgffb5w3ddx1ul22qy1wxspy5cpkz504,dharma pal,2015-05-29T02:30:18.971000,Nice song﻿,0
1,z123dbgb0mqjfxbtz22ucjc5jvzcv3ykj,Tiza Arellano,2015-05-29T00:14:48.748000,I love song ﻿,0
2,z12quxxp2vutflkxv04cihggzt2azl34pms0k,Prìñçeśś Âliś Łøvê Dømíñø Mâđiś™ ﻿,2015-05-28T21:00:08.607000,I love song ﻿,0
3,z12icv3ysqvlwth2c23eddlykyqut5z1h,Eric Gonzalez,2015-05-28T20:47:12.193000,"860,000,000 lets make it first female to reach...",0
4,z133stly3kete3tly22petvwdpmghrlli,Analena López,2015-05-28T17:08:29.827000,shakira is best for worldcup﻿,0


In [5]:
data.shape

(370, 5)

In [6]:
data.isnull().sum()  #should be 0 probs 

COMMENT_ID    0
AUTHOR        0
DATE          0
CONTENT       0
CLASS         0
dtype: int64

In [7]:
#dropping coliumns and namechange
data.drop(labels=['COMMENT_ID','AUTHOR','DATE',],axis=1,inplace=True)

In [8]:
#count no of words in each comment
data['Count']=0
for i in np.arange(0,len(data.CONTENT)):
    data.loc[i, 'Count'] = len(data.loc[i,'CONTENT'])
#values of class, should be 0 and 1
print("two values of class set: ", data.CLASS.unique())

two values of class set:  [0 1]


In [9]:
data.head()

,CONTENT,CLASS,Count
0,Nice song﻿,0,10
1,I love song ﻿,0,13
2,I love song ﻿,0,13
3,"860,000,000 lets make it first female to reach...",0,86
4,shakira is best for worldcup﻿,0,29


In [10]:
#aaaaaaaaaaaaaaa the boring work now
import re 
import string
import pandas as pd
def clean_text_round1(text):
    #makes lowercase, removes brackets, removes punct
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
r1 = lambda x: clean_text_round1(x)
    

In [11]:
data_clean = pd.DataFrame(data.CONTENT.apply(r1))
data_clean

,CONTENT
0,nice song﻿
1,i love song ﻿
2,i love song ﻿
3,lets make it first female to reach one billio...
4,shakira is best for worldcup﻿
...,...
365,i love this song because we sing it at camp al...
366,i love this song for two reasons is about afr...
367,wow
368,shakira u are so wiredo


In [12]:
# lets go againnnn aaaaaaaaaaaaaa
def clean_text_round2(text):
    #other punctuation, and non-sensical text missed
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text
r2 = lambda x: clean_text_round2(x)

In [13]:
data_clean = pd.DataFrame(data_clean.CONTENT.apply(r2))
data_clean

,CONTENT
0,nice song﻿
1,i love song ﻿
2,i love song ﻿
3,lets make it first female to reach one billio...
4,shakira is best for worldcup﻿
...,...
365,i love this song because we sing it at camp al...
366,i love this song for two reasons is about afr...
367,wow
368,shakira u are so wiredo


In [14]:
split = pd.DataFrame(data_clean.CONTENT.str.split(' '))
split

,CONTENT
0,"[nice, song﻿]"
1,"[i, love, song, ﻿]"
2,"[i, love, song, ﻿]"
3,"[, lets, make, it, first, female, to, reach, o..."
4,"[shakira, is, best, for, worldcup﻿]"
...,...
365,"[i, love, this, song, because, we, sing, it, a..."
366,"[i, love, this, song, for, two, reasons, , is,..."
367,[wow]
368,"[shakira, u, are, so, wiredo]"


In [15]:
data.head()

,CONTENT,CLASS,Count
0,Nice song﻿,0,10
1,I love song ﻿,0,13
2,I love song ﻿,0,13
3,"860,000,000 lets make it first female to reach...",0,86
4,shakira is best for worldcup﻿,0,29


In [16]:
data = data.assign(CONTENT=split['CONTENT'])
data.head()

,CONTENT,CLASS,Count
0,"[nice, song﻿]",0,10
1,"[i, love, song, ﻿]",0,13
2,"[i, love, song, ﻿]",0,13
3,"[, lets, make, it, first, female, to, reach, o...",0,86
4,"[shakira, is, best, for, worldcup﻿]",0,29


In [17]:
#stemming now
from nltk.stem.snowball import SnowballStemmer
#i am specifying english for optimisation (?) 
stemmer = SnowballStemmer("english")
data['S_CONTENT'] = data['CONTENT'].apply(lambda x: [stemmer.stem(y) for y in x])



In [18]:
from nltk.corpus import stopwords 
stops = set(stopwords.words("english"))

def remove_stopwords(n):
    list = n['S_CONTENT']
    useful = [w for w in list if not w in stops]
    return(useful)

data['STEMMED'] = data.apply(remove_stopwords, axis =1)
data['STR'] = data.STEMMED.apply(lambda x: ', '.join([str(i) for i in x]))


In [31]:
from sklearn.feature_extraction.text import CountVectorizer 
cv = CountVectorizer()
m = cv.fit_transform(data['STR'])
matrix = m.toarray()
print (matrix)
matrix.shape




[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


(370, 1067)